In [12]:
# Taken from main06.cc Pythia example. A study of event properties of LEP1 events

from numpythia import Pythia, hepmc_write, hepmc_read
from numpythia import STATUS, HAS_END_VERTEX, ABS_PDG_ID

from pyjet import cluster, DTYPE_PTEPM, DTYPE_EP
from pyjet.testdata import get_event

import numpy as np
import sys

from pyjet import cluster
from pyjet import ClusterSequenceArea, JetDefinition, ClusterSequence

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
# Pythia generator settings. Process selection e+e- collision.
# WeakSingleBoson:ffbar2gmZ : Produce Z bosons and interference with photon
# Consider only Z (PDGID=23) decays to light quarks

params = {"Beams:idA" : "11", "Beams:idB" : "-11","Beams:eCM": 91.1876, "WeakSingleBoson:ffbar2gmZ" : "on", "23:onMode": "off", "23:onIfAny": "1 2 3 4 5", "PDF:lepton": "off"}
pythia = Pythia(params =params, random_state=1)

# Consider only final-state particles
selection = ((STATUS == 1) & ~HAS_END_VERTEX)

In [3]:
# Array of pseudo-jets or particles to cluster
particles = []

# Begin event loop. Generate events
for event in pythia(events=1):
    particles = event.all(selection)

In [4]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('out.particles', 'w') as f:
    for p in particles:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %( p['px'], p['py'], p['pz'], p['E']))


In [5]:
# Cluster particles into jets
# First generate a whole "clustering sequence" with the e+e- kt algorithm

jetdef = JetDefinition('ee_kt')
sequence = ClusterSequence(particles, jetdef, ep=True)

In [8]:
# Then select the jets that come out of clustering 
jets = sequence.exclusive_jets(3)

In [62]:
with open('out.jetconst', 'w') as f:
    for c in range(len(jets[0].constituents_array(ep=True))):
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %(jets[0].constituents_array(ep=True)[c]['px'], jets[0].constituents_array(ep=True)[c]['py'], jets[0].constituents_array(ep=True)[c]['pz'], jets[0].constituents_array(ep=True)[c]['E']))

In [9]:
# Print out the jets to a file
with open('out.ktjets', 'w') as f:
    for jet in jets:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %(jet.px, jet.py, jet.pz, jet.e))

In [11]:
# Create qubo
from pyqubo import Spin, Array

# Create an array os spin variables
n_part = len(particles) 
s = Array.create('s', shape=n_part, vartype='BINARY')

# Create an array of zeroes for qubo matrix coefficients
coeff = [[0] * n_part for _ in range(n_part)] 

for i in range(0,n_part):
    for j in range(0,n_part):
        coeff[i][j] = (particles[i]['px']*particles[j]['px'] + particles[i]['py']*particles[j]['py'] + particles[i]['pz']*particles[j]['pz'] - particles[i]['E']*particles[j]['E'] * np.cos(np.pi/4.))/(1 - np.cos(np.pi/4))

# Construct Hamiltonian
H = sum([ coeff[i][j]*s[i]*s[j] for i in range(len(particles)) for j in range(len(particles))])

# Compile model using pyqubo

model = H.compile()
qubo, offset = model.to_qubo()

In [13]:
# Submit qubo to D-Wave

sampler = EmbeddingComposite(DWaveSampler(solver={'qpu':True}))
response = sampler.sample_qubo(qubo)

In [31]:
for sample in response:
    print(sample)

{'s[0]': 1, 's[10]': 1, 's[11]': 1, 's[12]': 0, 's[13]': 1, 's[14]': 1, 's[15]': 1, 's[16]': 1, 's[17]': 1, 's[18]': 1, 's[19]': 1, 's[1]': 1, 's[20]': 1, 's[21]': 0, 's[22]': 1, 's[23]': 1, 's[24]': 1, 's[25]': 0, 's[26]': 0, 's[27]': 0, 's[28]': 1, 's[29]': 1, 's[2]': 1, 's[30]': 1, 's[31]': 1, 's[32]': 1, 's[33]': 1, 's[34]': 0, 's[3]': 1, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 0}


In [32]:
solution = {'s[0]': 1, 's[10]': 1, 's[11]': 1, 's[12]': 0, 's[13]': 1, 's[14]': 1, 's[15]': 1, 's[16]': 1, 's[17]': 1, 's[18]': 1, 's[19]': 1, 's[1]': 1, 's[20]': 1, 's[21]': 0, 's[22]': 1, 's[23]': 1, 's[24]': 1, 's[25]': 0, 's[26]': 0, 's[27]': 0, 's[28]': 1, 's[29]': 1, 's[2]': 1, 's[30]': 1, 's[31]': 1, 's[32]': 1, 's[33]': 1, 's[34]': 0, 's[3]': 1, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 0}

In [46]:
with open('out.particlesqujet', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        if(solution[key]==1):
            f.write("0 0 0 0\n")
            f.write("%s %s %s %s\n\n\n" %( particles[i]['px'], particles[i]['py'], particles[i]['pz'], particles[i]['E']))

In [36]:
import pyqubo
decoded_solution, broken, energy = model.decode_solution(solution, vartype='BINARY')

AttributeError: 'cpp_pyqubo.Model' object has no attribute 'decode_solution'